In [2]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, widgets
import yfinance as yf

# Download historical stock data for a list of symbols
def download_stock_data(symbols):
    data = {}
    for symbol in symbols:
        stock = yf.Ticker(symbol)
        stock_data = stock.history(period='max')
        stock_data.reset_index(inplace=True)
        data[symbol] = stock_data
    return data

# Create an interactive 2D stock market plot
def create_stock_market_plot(data, symbols, date_range):
    fig = go.Figure()

    for symbol in symbols:
        # Convert the 'Date' column to timezone-naive
        data[symbol]['Date'] = data[symbol]['Date'].dt.tz_localize(None)
        # Convert date_range values to timezone-naive
        date_range_naive = (date_range[0].tz_localize(None), date_range[1].tz_localize(None))
        data_filtered = data[symbol][(data[symbol]['Date'] >= date_range_naive[0]) & (data[symbol]['Date'] <= date_range_naive[1])]
        fig.add_trace(go.Scatter(x=data_filtered['Date'], y=data_filtered['Close'],
                                 mode='lines', name=symbol, line_shape='linear'))

    fig.update_xaxes(title='Date', rangeslider=dict(visible=True), type='date')
    fig.update_yaxes(title='Close Price')

    fig.update_layout(title='Stock Market History',
                      margin=dict(r=10, l=10, b=10, t=30),
                      hovermode="x unified")

    return fig

# Display the plot for the specified stock symbols and date range
def display_plot(symbols_input, date_range):
    symbols = [symbol.strip() for symbol in symbols_input.split(',')]
    data = download_stock_data(symbols)
    fig = create_stock_market_plot(data, symbols, date_range)
    fig.show()

# Input widgets
symbols_input = widgets.Text(value='AAPL,MSFT,GOOGL', description='Symbols:')
date_range_slider = widgets.SelectionRangeSlider(
    options=pd.date_range(start='1900-01-01', end='2021-09-01', freq='M'),
    index=(0, len(pd.date_range(start='1900-01-01', end='2021-09-01', freq='M')) - 1),
    description='Date Range:',
    continuous_update=False
)

interact(display_plot, symbols_input=symbols_input, date_range=date_range_slider)


interactive(children=(Text(value='AAPL,MSFT,GOOGL', description='Symbols:'), SelectionRangeSlider(continuous_u…

<function __main__.display_plot(symbols_input, date_range)>